# `04-interstation-spend-time.ipynb`
앞에서 만든 데이터를 통해서 한 역에서 배차 간격 구하기.

In [2]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
import pandas as pd
from tqdm import tqdm
import util

In [4]:
DAYTYPE = ["토요일", "일요일", "공휴일", "평일"]
PATH_CUSTOMER_IO = "../raw_data/selected/부산교통공사_시간대별 승하차인원.csv"
PATH_STATION_SCHEDULE_SEPERATED_DAYTYPE = util.PATH_STATION_SCHEDULE_SEPERATED_DAYTYPE
PATH_SCHEDULE_INTERVAL_MEAN = [
    f"../trimmed_data/ScheduleIntervalMean{day}.pickle" for day in DAYTYPE
]

## Weight ready


In [5]:
# 가중합의 가중치를 구하기 위한 데이터 불러오기
# 가중치는 각 시간대 별로 인원수를 가중치로 함
df_weight = pd.read_csv(PATH_CUSTOMER_IO, encoding="EUC-KR")
df_weight_SUM = df_weight.loc[:, ["역번호", "합계"]].groupby("역번호").sum()
target_columns = df_weight.columns[6:]
print(df_weight.columns)
df_weight.head(3)

Index(['역번호', '역명', '년월일', '요일', '구분', '합계', '01시-02시', '02시-03시', '03시-04시',
       '04시-05시', '05시-06시', '06시-07시', '07시-08시', '08시-09시', '09시-10시',
       '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시', '15시-16시',
       '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시',
       '22시-23시', '23시-24시', '24시-01시'],
      dtype='object')


,역번호,역명,년월일,요일,구분,합계,01시-02시,02시-03시,03시-04시,04시-05시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시-24시,24시-01시
0,95,다대포해수욕장,2024-01-01,월,승차,8587,0,0,0,12,...,585,537,468,266,162,84,111,70,16,0
1,95,다대포해수욕장,2024-01-01,월,하차,8377,0,4,0,1,...,363,369,303,192,127,92,97,54,42,31
2,95,다대포해수욕장,2024-01-02,화,승차,4882,0,1,1,16,...,458,422,374,221,142,85,72,50,14,0


In [6]:
# 참고: GroupBy 때문에 역번호가 자동으로 INDEX로 변함.
# TODO : 시간에 해당하는 것만 합쳐서 가져오기
df_weight = (
    df_weight.iloc[:, [0, *range(6, len(df_weight.columns))]].groupby("역번호").sum()
)

# n~m시를 n 으로 바꾸기
time_to_integer = {
    curr: new_name for curr, new_name in zip(target_columns, list(range(1, 25)))
}
df_weight = df_weight.rename(columns=time_to_integer)

# TODO: 가중치로 만들고 value를 가중치로 다시 바꾸기.
for index in tqdm(df_weight_SUM.index):
    df_weight.loc[index, :] = df_weight.loc[index, :] / df_weight_SUM.loc[index, "합계"]

# TODO: 환승역 중복이 제거 되어 있어서 다시 추가함
# 어캐알았음? -> 에러나서 삽질함.
df_weight.loc[301, :] = df_weight.loc[208, :]
df_weight.loc[401, :] = df_weight.loc[309, :]

df_weight.head(3)

100%|██████████| 112/112 [00:00<00:00, 2793.58it/s]


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
역번호,,,,,,,,,,,,,,,,,,,,,
95,0.000065,0.000044,0.000044,0.002301,0.009177,0.025826,0.052917,0.065470,0.051581,0.056159,...,0.085282,0.080876,0.078471,0.067389,0.041138,0.027968,0.023160,0.02016,0.011687,0.006221
96,0.000033,0.000026,0.000002,0.001308,0.010910,0.025226,0.046469,0.065929,0.055753,0.056943,...,0.079309,0.083350,0.080991,0.076191,0.043655,0.030881,0.026743,0.02004,0.009715,0.003944
97,0.000054,0.000047,0.000018,0.001577,0.011281,0.025697,0.052684,0.074127,0.056272,0.056377,...,0.069604,0.074801,0.080825,0.080666,0.049541,0.033859,0.031175,0.02537,0.014503,0.005464


## Weghted Sum Calculate step

### Weighted Sum ready Processing
간단히 프로세싱을 보고 싶으면 아래의 example session을 참고

In [7]:
def get_weigthed_mean(path,weight=df_weight):
    import util
    dat = pd.read_json(path, encoding="UTF-8")
    
    __dat = (
        dat.iloc[:, 2:6]
        .sort_values(by=["StationList", "DirectionVector", "ArriveSchedule"])
        .__deepcopy__()
    )
    
    __index = __dat.groupby(by=["DirectionVector", "StationList"])
    __index = __index.mean().index
    
    __hour = dat.iloc[:, 4:-1] // 60
    __hour["DirectionVector"] = dat["DirectionVector"]
    __hour["StationList"] = dat["StationList"]
    __weight_table = weight
    __weight_table = __weight_table.T.to_dict()


    for d_vec, s_code in tqdm(__index):
        condtion_df = (__hour["DirectionVector"] == d_vec) & (__hour["StationList"] == s_code)
        __hour.loc[condtion_df, "ArriveSchedule"] = __hour.loc[
            condtion_df, "ArriveSchedule"
        ].replace(__weight_table[s_code])
        __hour.loc[condtion_df, "DepartSchedule"] = __hour.loc[
            condtion_df, "DepartSchedule"
        ].replace(__weight_table[s_code])

    __weight = __hour.rename(
        columns={"ArriveSchedule": "ArriveWeight", "DepartSchedule": "DepartWeight"}
    )
    
    __diff = __dat.copy()
    for d_vec, s_code in tqdm(__index):
        condtion_df = (__diff["DirectionVector"] == d_vec) & (__diff["StationList"] == s_code)
        __diff.loc[condtion_df, "ArriveSchedule"] = __diff.loc[
            condtion_df, "ArriveSchedule"
        ].diff()
        __diff.loc[condtion_df, "DepartSchedule"] = __diff.loc[
            condtion_df, "DepartSchedule"
        ].diff()

    __diff = __diff.rename(
        columns={"ArriveSchedule": "ArriveInterval", "DepartSchedule": "DepartInterval"}
    )
    
    __weight = __weight.reindex()
    __diff = __diff.reindex()
    operation_target = {"ArriveInterval": "ArriveWeight", "DepartInterval": "DepartWeight"}
    __weighted_interval = util.df_mul_DiffColName(__weight,__diff,operation_target)

    __weighted_interval_grouped = __weighted_interval.groupby(by=["DirectionVector","StationList"]).sum()
    __weight_sum_grouped = __weight.groupby(by=["DirectionVector","StationList"]).sum()
    
    # import sys;sys.exit()
    return util.df_div_DiffColName(__weighted_interval_grouped,__weight_sum_grouped,operation_target)

# 요일 별로 처리하기.
for loadPath,saveFile in zip(PATH_STATION_SCHEDULE_SEPERATED_DAYTYPE,PATH_SCHEDULE_INTERVAL_MEAN):
    weighted_interval_mean = get_weigthed_mean(loadPath)
    print(weighted_interval_mean.head(2))
    util.savePickle(weighted_interval_mean,saveFile)

100%|██████████| 228/228 [00:00<00:00, 737.26it/s]


                             ArriveInterval  DepartInterval
DirectionVector StationList                                
0               95                 6.052231        6.052231
                96                 6.078447        6.076206


100%|██████████| 28/28 [00:00<00:00, 805.35it/s]


                             ArriveInterval  DepartInterval
DirectionVector StationList                                
6               401                7.813510             NaN
                402                7.817525         7.81886


100%|██████████| 200/200 [00:00<00:00, 745.86it/s]


                             ArriveInterval  DepartInterval
DirectionVector StationList                                
0               95                 6.331693        6.341629
                96                 6.362998        6.359737


100%|██████████| 228/228 [00:00<00:00, 766.24it/s]


                             ArriveInterval  DepartInterval
DirectionVector StationList                                
0               95                 6.074339        6.074339
                96                 6.064703        6.062306


### Calculate Weighted SUM 역마다 가중 평균 배차간격 구하기

In [8]:
path = PATH_SCHEDULE_INTERVAL_MEAN

# import sys;sys.exit()
# 6일 평균.
tmp = util.loadPickle(path[3]).mean(axis=1) * 5  # 평일
tmp += util.loadPickle(path[0]).mean(axis=1)  # 토요일
tmp /= 6
tmp = tmp.groupby(by="StationList").mean()
tmp *= 60  # Time Unit is Second!!

PATH_WEIGHTED_MEAN_INTERVAL = "../trimmed_data/WeightedIntervalMean.pickle"
util.savePickle(tmp, PATH_WEIGHTED_MEAN_INTERVAL)
(tmp[95],tmp[201],tmp[301],tmp[401])

(355.9588973127273, 363.9968771614468, 392.5459131354582, 404.6896758014144)

### Example Session
---
이곳부터는 실행되는 자세한 코드 상태를 보여줍니다

In [9]:
# Load Data
path = PATH_STATION_SCHEDULE_SEPERATED_DAYTYPE[0]
dat = pd.read_json(path, encoding="UTF-8")
dat.tail(3)

,TrainCode,LineNumber,DirectionVector,StationList,ArriveSchedule,DepartSchedule,DayType
36435,4296,4,7,412,1481,1481.0,토요일
36436,4296,4,7,413,1482,1483.0,토요일
36437,4296,4,7,414,1484,NaN,토요일


In [10]:
# Get INDEX
# INDEX의 타입은 MUTI INDEX 입니다.
# 멀티인덱스는 순서쌍으로 Duple로 표현됩니다.
__dat = (
    dat.iloc[:, 2:6]
    .sort_values(by=["StationList", "DirectionVector", "ArriveSchedule"])
    .__deepcopy__()
)
print("\n__dat 예시 :")
print(__dat.head(3))

__index = __dat.groupby(by=["DirectionVector", "StationList"])
__index = __index.mean().index
print("\n__index 예시 :")
print(*__index[:3], len(__index), sep="  ")


__dat 예시 :
      DirectionVector  StationList  ArriveSchedule  DepartSchedule
6640                0           95             304           304.0
6680                0           95             318           318.0
6720                0           95             330           330.0

__index 예시 :
(0, 95)  (0, 96)  (0, 97)  228


In [11]:
# Hours에 따른 LookUp Table을 weight와 매칭 해준다.
__hour = dat.iloc[:, 4:-1] // 60
__hour["DirectionVector"] = dat["DirectionVector"]
__hour["StationList"] = dat["StationList"]
__weight_table = df_weight
__weight_table = __weight_table.T.to_dict()


for d_vec, s_code in tqdm(__index):
    condtion_df = (__hour["DirectionVector"] == d_vec) & (__hour["StationList"] == s_code)
    __hour.loc[condtion_df, "ArriveSchedule"] = __hour.loc[
        condtion_df, "ArriveSchedule"
    ].replace(__weight_table[s_code])
    __hour.loc[condtion_df, "DepartSchedule"] = __hour.loc[
        condtion_df, "DepartSchedule"
    ].replace(__weight_table[s_code])

__weight = __hour.rename(
    columns={"ArriveSchedule": "ArriveWeight", "DepartSchedule": "DepartWeight"}
)

print("\n__hour 예시 :")
print(__weight.head(3), len(__weight))

  0%|          | 0/228 [00:00<?, ?it/s]

100%|██████████| 228/228 [00:00<00:00, 296.86it/s]


__hour 예시 :
   ArriveWeight  DepartWeight  DirectionVector  StationList
0      0.003707      0.003707                1          134
1      0.006896      0.006896                1          133
2      0.009026      0.009026                1          132 36438


In [12]:
# Schedule Difference 구하기
__diff = __dat.copy()
for d_vec, s_code in tqdm(__index):
    condtion_df = (__diff["DirectionVector"] == d_vec) & (__diff["StationList"] == s_code)
    __diff.loc[condtion_df, "ArriveSchedule"] = __diff.loc[
        condtion_df, "ArriveSchedule"
    ].diff()
    __diff.loc[condtion_df, "DepartSchedule"] = __diff.loc[
        condtion_df, "DepartSchedule"
    ].diff()

__diff = __diff.rename(
    columns={"ArriveSchedule": "ArriveInterval", "DepartSchedule": "DepartInterval"}
)

print("\n__diff 예시 :")
print(__diff.head(3), len(__diff))
    # import sys; sys.exit()

100%|██████████| 228/228 [00:00<00:00, 753.57it/s]


__diff 예시 :
      DirectionVector  StationList  ArriveInterval  DepartInterval
6640                0           95             NaN             NaN
6680                0           95            14.0            14.0
6720                0           95            12.0            12.0 36438


In [13]:
import util
__weight = __weight.reindex()
__diff = __diff.reindex()
operation_target = {"ArriveInterval": "ArriveWeight", "DepartInterval": "DepartWeight"}
__weighted_interval = util.df_mul_DiffColName(__weight,__diff,operation_target)

# Weighted Sum
__weighted_interval_grouped = __weighted_interval.groupby(by=["DirectionVector","StationList"]).sum()
__weight_sum_grouped = __weight.groupby(by=["DirectionVector","StationList"]).sum()
util.df_div_DiffColName(__weighted_interval_grouped,__weight_sum_grouped,operation_target).tail(3)

ArriveInterval  DepartInterval
DirectionVector StationList                                
7               412                7.218161        7.219375
                413                7.137433        7.137013
                414                7.163189             NaN